In [1]:
import numpy as np
import tensorflow as tf

from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Convolution2D, AveragePooling2D
from keras.layers import Input, Activation, Lambda
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.applications.resnet50 import identity_block, conv_block
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras import optimizers


Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5110)
/home/mzhao/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
px_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def preprocess(x):
    x = x - px_mean
    return x[:, ::-1]
#     return tf.reverse(x, [False, True, False, False])

size = (270, 480)
n_classes = 8
lr = 0.001
batch_size = 64
dropout = 0.4
weights_file = 'vgg16.h5'

model = Sequential()
model.add(Lambda(preprocess, input_shape=(3,) + size))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1000, activation='softmax'))

model.load_weights(weights_file)
model.pop(); model.pop(); model.pop()

for layer in model.layers:
    layer.trainable = False

model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adadelta", metrics=["accuracy"])

In [6]:
def inception_preprocess(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

size = (299, 299)
n_classes = 8
lr = 0.001
batch_size = 64

img_input = Input(shape=(3, size[0], size[1]))
inception = InceptionV3(include_top=False, weights='imagenet', input_tensor=img_input)

for layer in inception.layers:
    layer.trainable = False

output = inception.output
output = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(output)
output = Flatten(name='flatten')(output)
output = Dense(n_classes, activation='softmax', name='predictions')(output)

model = Model(inception.input, output)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9, decay=0.0, nesterov=True),
              metrics=["accuracy"])

In [7]:
trn_path = 'train/'
train_datagen = ImageDataGenerator()
trn_gen = train_datagen.flow_from_directory(trn_path, target_size=size, batch_size=50,
                                                    class_mode='categorical', shuffle=True)

# model.fit(X_train, Y_train, nb_epoch=5, batch_size=32)
# loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)

Found 3777 images belonging to 8 classes.


In [6]:
# inception net
nb_trn_samples = 3777
nb_epoch = 20

model.fit_generator(trn_gen, samples_per_epoch=nb_trn_samples, nb_epoch=nb_epoch, verbose=2)

Epoch 1/10
80s - loss: 2.1645 - acc: 0.1660
Epoch 2/10
79s - loss: 1.9904 - acc: 0.2568
Epoch 3/10
79s - loss: 1.8615 - acc: 0.3272
Epoch 4/10
79s - loss: 1.7833 - acc: 0.3691
Epoch 5/10
79s - loss: 1.7187 - acc: 0.4069
Epoch 6/10
79s - loss: 1.6561 - acc: 0.4440
Epoch 7/10
79s - loss: 1.6144 - acc: 0.4676
Epoch 8/10
79s - loss: 1.5815 - acc: 0.4739
Epoch 9/10
79s - loss: 1.5394 - acc: 0.4951
Epoch 10/10
79s - loss: 1.5039 - acc: 0.5068


In [8]:
# vgg
nb_trn_samples = 3777
nb_epoch = 20

model.fit_generator(trn_gen, samples_per_epoch=nb_trn_samples, nb_epoch=nb_epoch, verbose=2)

Epoch 1/20


RuntimeError: Cuda error: GpuElemwise node_887ef546c8e7d58521624ceffe59f12e_0 Composite: out of memory.
    n_blocks=30 threads_per_block=256
   Call: kernel_Composite_node_887ef546c8e7d58521624ceffe59f12e_0_Ccontiguous<<<n_blocks, threads_per_block>>>(numEls, local_dims[0], local_dims[1], local_dims[2], i0_data, local_str[0][0], local_str[0][1], local_str[0][2], i1_data, local_str[1][0], local_str[1][1], local_str[1][2], o0_data, local_ostr[0][0], local_ostr[0][1], local_ostr[0][2])

Apply node that caused the error: GpuElemwise{Composite{((i0 + i1) + Abs((i0 + i1)))},no_inplace}(GpuDnnConv{algo='small', inplace=True}.0, GpuReshape{4}.0)
Toposort index: 1245
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, (True, False, True, True))]
Inputs shapes: [(50, 32, 149, 149), (1, 32, 1, 1)]
Inputs strides: [(710432, 22201, 149, 1), (0, 1, 0, 0)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuCAReduce{add}{1,0,1,1}(GpuElemwise{Composite{((i0 + i1) + Abs((

In [19]:
# paths
train_path = '/train/'
# test_path = '/test/'
saved_model_path = '/models/'

# data
batch_size = 16
nb_split_train_samples = 3277
nb_full_train_samples = 3785
nb_valid_samples = 500
# nb_test_samples = 1000
classes = ["ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

# model
nb_runs = 5
nb_epoch = 30
nb_aug = 5
dropout = 0.4
clip = 0.01
use_val = False
archs = ["vggbn"]

models = {
    "vggbn": Vgg16BN(size=(270, 480), n_classes=nb_classes, lr=0.001,
                           batch_size=batch_size, dropout=dropout),
#     "inception": Inception(size=(299, 299), n_classes=nb_classes,
#                            lr=0.001, batch_size=batch_size),
#     "resnet": Resnet50(size=(270, 480), n_classes=nb_classes, lr=0.001,
#                     batch_size=batch_size, dropout=dropout)
}